In [1]:
!git clone https://github.com/wchill/HMP_Dataset.git

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200407102721-0000
KERNEL_ID = 1d439fdf-0247-4f34-931c-fbb819dabfb6
Cloning into 'HMP_Dataset'...
remote: Enumerating objects: 865, done.
remote: Total 865 (delta 0), reused 0 (delta 0), pack-reused 865
Receiving objects: 100% (865/865), 1010.96 KiB | 0 bytes/s, done.
Checking out files: 100% (848/848), done.


In [21]:
#check the content of the folder HMP_Dataset
!ls HMP_Dataset

Brush_teeth	Drink_glass  Liedown_bed  Sitdown_chair  final.py
Climb_stairs	Eat_meat     MANUAL.txt   Standup_chair  impdata.py
Comb_hair	Eat_soup     Pour_water   Use_telephone
Descend_stairs	Getup_bed    README.txt   Walk


In [4]:
!ls HMP_Dataset/Brush_teeth

os.listdir('HMP_Dataset/Brush_teeth')

Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt
Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt
Accelerometer-2011-05-30-08-35-11-brush_teeth-f1.txt
Accelerometer-2011-05-30-09-36-50-brush_teeth-f1.txt
Accelerometer-2011-05-30-10-34-16-brush_teeth-m1.txt
Accelerometer-2011-05-30-21-10-57-brush_teeth-f1.txt
Accelerometer-2011-05-30-21-55-04-brush_teeth-m2.txt
Accelerometer-2011-05-31-15-16-47-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-42-22-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-45-50-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-45-27-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-48-05-brush_teeth-f1.txt


['Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt',
 'Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt',
 'Accelerometer-2011-05-30-08-35-11-brush_teeth-f1.txt',
 'Accelerometer-2011-05-30-09-36-50-brush_teeth-f1.txt',
 'Accelerometer-2011-05-30-10-34-16-brush_teeth-m1.txt',
 'Accelerometer-2011-05-30-21-10-57-brush_teeth-f1.txt',
 'Accelerometer-2011-05-30-21-55-04-brush_teeth-m2.txt',
 'Accelerometer-2011-05-31-15-16-47-brush_teeth-f1.txt',
 'Accelerometer-2011-06-02-10-42-22-brush_teeth-f1.txt',
 'Accelerometer-2011-06-02-10-45-50-brush_teeth-f1.txt',
 'Accelerometer-2011-06-06-10-45-27-brush_teeth-f1.txt',
 'Accelerometer-2011-06-06-10-48-05-brush_teeth-f1.txt']

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType

## Create a data structure or the data schema 

schema = StructType([
    StructField('x', IntegerType(), True),
    StructField('y', IntegerType(), True),
    StructField('z', IntegerType(), True)
])


In [6]:
import os
 
## extract content of the main folder 
file_list = os.listdir('HMP_Dataset')
file_list

['.git',
 '.idea',
 'Brush_teeth',
 'Climb_stairs',
 'Comb_hair',
 'Descend_stairs',
 'Drink_glass',
 'Eat_meat',
 'Eat_soup',
 'Getup_bed',
 'Liedown_bed',
 'MANUAL.txt',
 'Pour_water',
 'README.txt',
 'Sitdown_chair',
 'Standup_chair',
 'Use_telephone',
 'Walk',
 'final.py',
 'impdata.py']

In [7]:
## Select only dataset classes (folder names) and filter files
file_list_filtred = [s for s in file_list if '_' in s]

In [8]:
file_list_filtred

['Brush_teeth',
 'Climb_stairs',
 'Comb_hair',
 'Descend_stairs',
 'Drink_glass',
 'Eat_meat',
 'Eat_soup',
 'Getup_bed',
 'Liedown_bed',
 'Pour_water',
 'Sitdown_chair',
 'Standup_chair',
 'Use_telephone']

In [9]:
## Load data 


## Initialize an empty data frame
df = None


from pyspark.sql.functions import lit

## Read the content (files) of each folder
for category in file_list_filtred:
    data_files = os.listdir('HMP_Dataset/'+category)
    
    ## read the content of each file in a temporary dataframe (temp_df), each file contains data structred as the defined schema (x, y, z)
    for data_file in data_files:
        ##print(data_file)
        temp_df = spark.read.option("header","false").option("delimiter", " ").csv('HMP_Dataset/'+category+'/'+data_file, schema = schema)
        
        ## Add a two columns 'class' containing the category and 'source' containing the file name
        temp_df = temp_df.withColumn('class',lit(category))
        temp_df = temp_df.withColumn('source',lit(data_file))
        
        if df is None:
            df = temp_df
        else:
            df = df.union(temp_df)

In [10]:

df.show()

+---+---+---+-----------+--------------------+
|  x|  y|  z|      class|              source|
+---+---+---+-----------+--------------------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|
| 22| 51| 34|Brush_teeth|Accelerometer-201...|
| 20| 50| 35|Brush_teeth|Accelerometer-201...|
| 22| 52| 34|Brush_teeth|Accelerometer-201...|
| 22| 50| 34|Brush_teeth|Accelerometer-201...|
| 22| 51| 35|Brush_teeth|Accelerometer-201...|
| 21| 51| 33|Brush_teeth|Accelerometer-201...|
| 20| 50| 34|Brush_teeth|Accelerometer-201...|
| 21| 49| 33|Brush_teeth|Accelerometer-201...|
| 21| 49| 33|Brush_teeth|Accelerometer-201...|
| 20| 51| 35|Brush_teeth|Accelerometer-201...|
| 18| 49| 34|Brush_teeth|Accelerometer-201...|
| 19| 48| 34|Brush_teeth|Accelerometer-201...|
| 16| 53| 34|Brush_teeth|Accelerometer-201...|
| 18| 52| 35|

In [11]:
## Transform Data

## Crate a STRING INDEXER in order to transform string classes to a number of integers inorder to permit to machine learning algorithms to cope with it
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="class",outputCol="classIndexer")

indexed = indexer.fit(df).transform(df)

indexed.show()

+---+---+---+-----------+--------------------+------------+
|  x|  y|  z|      class|              source|classIndexer|
+---+---+---+-----------+--------------------+------------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|         5.0|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|         5.0|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|         5.0|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|         5.0|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|         5.0|
| 22| 51| 34|Brush_teeth|Accelerometer-201...|         5.0|
| 20| 50| 35|Brush_teeth|Accelerometer-201...|         5.0|
| 22| 52| 34|Brush_teeth|Accelerometer-201...|         5.0|
| 22| 50| 34|Brush_teeth|Accelerometer-201...|         5.0|
| 22| 51| 35|Brush_teeth|Accelerometer-201...|         5.0|
| 21| 51| 33|Brush_teeth|Accelerometer-201...|         5.0|
| 20| 50| 34|Brush_teeth|Accelerometer-201...|         5.0|
| 21| 49| 33|Brush_teeth|Accelerometer-201...|         5.0|
| 21| 49| 33|Brush_teeth|Accelerometer-2

In [13]:
## We use ONE-HOT-ENCODER inorder to transform a single column number value to a vector containing 12 columns and with 11 zeros and one column contain 
## "one" 
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCol="classIndexer",outputCol="categoryVec")

encoded = encoder.transform(indexed)

## , you see here, we have created categoryvec and this is the one-hot encoded vector,So, it's a compressed vector that means here, this 12 means it
## has 12 elements and it says here at position three that is a one. So, if you would have a second one index sparse vector at position five, then it
## would look like the following. 
encoded.show()

+---+---+---+-----------+--------------------+------------+--------------+
|  x|  y|  z|      class|              source|classIndexer|   categoryVec|
+---+---+---+-----------+--------------------+------------+--------------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|
| 22| 51| 34|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|
| 20| 50| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|
| 22| 52| 34|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|
| 22| 50| 34|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|
| 22| 51| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|
| 21| 51| 33|Brush_teeth|

In [14]:
##We have to transform our values X, Y, Z into vectors, because Spark ML only can work on vector objects so let's actually do that. So, let's import 
## vectors and the vector assembler. 

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

VectorAssembler = VectorAssembler(inputCols=['x','y','z'],outputCol='features')

features_vectorized = VectorAssembler.transform(encoded)

features_vectorized.show()

+---+---+---+-----------+--------------------+------------+--------------+----------------+
|  x|  y|  z|      class|              source|classIndexer|   categoryVec|        features|
+---+---+---+-----------+--------------------+------------+--------------+----------------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,49.0,35.0]|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,49.0,35.0]|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,52.0,35.0]|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,52.0,35.0]|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[21.0,52.0,34.0]|
| 22| 51| 34|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,51.0,34.0]|
| 20| 50| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[20.0,50.0,35.0]|
| 22| 52| 34|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,

In [15]:
from pyspark.ml.feature import Normalizer

Normalizer = Normalizer(inputCol='features', outputCol='features_norm')

normalized = Normalizer.transform(features_vectorized)

normalized.show()

+---+---+---+-----------+--------------------+------------+--------------+----------------+--------------------+
|  x|  y|  z|      class|              source|classIndexer|   categoryVec|        features|       features_norm|
+---+---+---+-----------+--------------------+------------+--------------+----------------+--------------------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,49.0,35.0]|[0.34316403829308...|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,49.0,35.0]|[0.34316403829308...|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,52.0,35.0]|[0.33117360613218...|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,52.0,35.0]|[0.33117360613218...|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[21.0,52.0,34.0]|[0.32020976616922...|
| 22| 51| 34|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,51.0,34.0]|[0.33

In [18]:

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer,encoder,VectorAssembler,Normalizer])
model = pipeline.fit(df)
prediction = model.transform(df)

In [19]:
prediction.show()

+---+---+---+-----------+--------------------+------------+--------------+----------------+--------------------+
|  x|  y|  z|      class|              source|classIndexer|   categoryVec|        features|       features_norm|
+---+---+---+-----------+--------------------+------------+--------------+----------------+--------------------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,49.0,35.0]|[0.34316403829308...|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,49.0,35.0]|[0.34316403829308...|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,52.0,35.0]|[0.33117360613218...|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,52.0,35.0]|[0.33117360613218...|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[21.0,52.0,34.0]|[0.32020976616922...|
| 22| 51| 34|Brush_teeth|Accelerometer-201...|         5.0|(12,[5],[1.0])|[22.0,51.0,34.0]|[0.33

In [20]:
df_train = prediction.drop("source").drop("class").drop("classIndex").drop("features").drop("x").drop("y").drop("z")

+------------+--------------+--------------------+
|classIndexer|   categoryVec|       features_norm|
+------------+--------------+--------------------+
|         5.0|(12,[5],[1.0])|[0.34316403829308...|
|         5.0|(12,[5],[1.0])|[0.34316403829308...|
|         5.0|(12,[5],[1.0])|[0.33117360613218...|
|         5.0|(12,[5],[1.0])|[0.33117360613218...|
|         5.0|(12,[5],[1.0])|[0.32020976616922...|
|         5.0|(12,[5],[1.0])|[0.33782247905081...|
|         5.0|(12,[5],[1.0])|[0.31139957766460...|
|         5.0|(12,[5],[1.0])|[0.33379342096892...|
|         5.0|(12,[5],[1.0])|[0.34191842968811...|
|         5.0|(12,[5],[1.0])|[0.33510742122295...|
|         5.0|(12,[5],[1.0])|[0.32673201960653...|
|         5.0|(12,[5],[1.0])|[0.31403714651066...|
|         5.0|(12,[5],[1.0])|[0.33494058369652...|
|         5.0|(12,[5],[1.0])|[0.33494058369652...|
|         5.0|(12,[5],[1.0])|[0.30765590086407...|
|         5.0|(12,[5],[1.0])|[0.28893535304210...|
|         5.0|(12,[5],[1.0])|[0

In [22]:
df_train.show()

+------------+--------------+--------------------+
|classIndexer|   categoryVec|       features_norm|
+------------+--------------+--------------------+
|         5.0|(12,[5],[1.0])|[0.34316403829308...|
|         5.0|(12,[5],[1.0])|[0.34316403829308...|
|         5.0|(12,[5],[1.0])|[0.33117360613218...|
|         5.0|(12,[5],[1.0])|[0.33117360613218...|
|         5.0|(12,[5],[1.0])|[0.32020976616922...|
|         5.0|(12,[5],[1.0])|[0.33782247905081...|
|         5.0|(12,[5],[1.0])|[0.31139957766460...|
|         5.0|(12,[5],[1.0])|[0.33379342096892...|
|         5.0|(12,[5],[1.0])|[0.34191842968811...|
|         5.0|(12,[5],[1.0])|[0.33510742122295...|
|         5.0|(12,[5],[1.0])|[0.32673201960653...|
|         5.0|(12,[5],[1.0])|[0.31403714651066...|
|         5.0|(12,[5],[1.0])|[0.33494058369652...|
|         5.0|(12,[5],[1.0])|[0.33494058369652...|
|         5.0|(12,[5],[1.0])|[0.30765590086407...|
|         5.0|(12,[5],[1.0])|[0.28893535304210...|
|         5.0|(12,[5],[1.0])|[0

In [23]:
df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- z: integer (nullable = true)
 |-- class: string (nullable = false)
 |-- source: string (nullable = false)



In [25]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-ea077ea4-9c80-49c6-86bd-a1850ae92a5d',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'api_key': 'Hqn4G20dLSvKzwoY8zLu_RVJ5tguL9LZa6cVcvna3TY4'
}

configuration_name = 'os_31daaa1631b34a279477ff52936c86c0_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

# df_data_1 = spark.read.json(cos.url('floorsensordata2604.json', 'sparktuto-donotdelete-pr-jsw0wchjtrvwr5'))
# df_data_1.take(5)

# we repartition it to one partition in order to make sure that we only create one file.
df = df.repartition(1)
df.write.parquet(cos.url('test.parquet', 'sparktuto-donotdelete-pr-jsw0wchjtrvwr5'))


In [26]:
# let's actually load the data frame from object store, just to make sure that everything works.
# So it's spark.read.parquet and then URL, so we copy it from here.

df_from_object_store = spark.read.parquet(cos.url('test.parquet', 'sparktuto-donotdelete-pr-jsw0wchjtrvwr5'))

# We notice that reading the DF from the object store is faster than loading the data file from desktop to 
# to the object store

In [27]:
df_from_object_store.show()

+---+---+---+-----------+--------------------+
|  x|  y|  z|      class|              source|
+---+---+---+-----------+--------------------+
| 22| 49| 35|Brush_teeth|Accelerometer-201...|
| 22| 49| 35|Brush_teeth|Accelerometer-201...|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|
| 22| 52| 35|Brush_teeth|Accelerometer-201...|
| 21| 52| 34|Brush_teeth|Accelerometer-201...|
| 22| 51| 34|Brush_teeth|Accelerometer-201...|
| 20| 50| 35|Brush_teeth|Accelerometer-201...|
| 22| 52| 34|Brush_teeth|Accelerometer-201...|
| 22| 50| 34|Brush_teeth|Accelerometer-201...|
| 22| 51| 35|Brush_teeth|Accelerometer-201...|
| 21| 51| 33|Brush_teeth|Accelerometer-201...|
| 20| 50| 34|Brush_teeth|Accelerometer-201...|
| 21| 49| 33|Brush_teeth|Accelerometer-201...|
| 21| 49| 33|Brush_teeth|Accelerometer-201...|
| 20| 51| 35|Brush_teeth|Accelerometer-201...|
| 18| 49| 34|Brush_teeth|Accelerometer-201...|
| 19| 48| 34|Brush_teeth|Accelerometer-201...|
| 16| 53| 34|Brush_teeth|Accelerometer-201...|
| 18| 52| 35|